# Import necessary dependencies and settings

In [2]:
# importa pandas y numpy
import pandas as pd
import numpy as np


# Transforming Nominal Features

Nominal attributes consist of discrete categorical values with no notion or sense of order amongst them. The idea here is to transform these attributes into a more representative numerical format which can be easily understood by downstream code and pipelines. Let’s look at a new dataset pertaining to video game sales.

In [3]:
# lee 'vgsales.csv'
# muestra las primeras 6 filas de las columnas 'Name', 'Platform', 'Year', 'Genre', 'Publisher'

vg_df = pd.read_csv("Ficheros/vgsales.csv", encoding = "utf-8")
vg_df[['Name', 'Platform', "Year", "Genre", "Publisher"]].iloc[0:7]

,Name,Platform,Year,Genre,Publisher
0,Wii Sports,Wii,2006.0,Sports,Nintendo
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo
5,Tetris,GB,1989.0,Puzzle,Nintendo
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo


### Get the list of unique video game genres 

In [4]:
generos = vg_df.Genre.unique()
generos

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy'], dtype=object)

This tells us that we have 12 distinct video game genres. 

### We can now generate a label encoding scheme for mapping each category to a numeric value by leveraging scikit-learn LabelEncoder

In [5]:
# usando LabelEncoder muestra los géneros y las categorías asociadas a cada género
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(generos)

le.classes_


array(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'], dtype=object)

In [6]:
le.transform(le.classes_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

### Show the transformed labels values and the dataframe

In [7]:
# primero muestra solo los géneros del DataFrame
vg_df.Genre

0              Sports
1            Platform
2              Racing
3              Sports
4        Role-Playing
             ...     
16593        Platform
16594         Shooter
16595          Racing
16596          Puzzle
16597        Platform
Name: Genre, Length: 16598, dtype: object

In [8]:
# muestra en el DataFrame los géneros y sus categorías asociadas 
vg_df['Genero_OneHot_Encoder'] = le.transform(vg_df.Genre)
vg_df.loc[:, ['Genre', 'Genero_OneHot_Encoder']].head(10)


,Genre,Genero_OneHot_Encoder
0,Sports,10
1,Platform,4
2,Racing,6
3,Sports,10
4,Role-Playing,7
5,Puzzle,5
6,Platform,4
7,Misc,3
8,Platform,4
9,Shooter,8



# Transforming Ordinal Features

Ordinal attributes are categorical attributes with a sense of order amongst the values. Let’s consider the Pokémon dataset. Let’s focus more specifically on the Type 1 attribute. We will think that each Type 1 has a different power that we can order.


In [9]:
# lee Pokemon.csv y muestra un head()
poke_df = pd.read_csv("Ficheros/Pokemon.csv", encoding = 'latin1', index_col = 0)
poke_df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary
#,,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,2,False
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,3,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,2,False


In [10]:
# usa un sample() con semilla 1 y toma todo el DataFrame para desordenarlo aleatoriamente
# resetea los índices y haz un head()

poke_df = poke_df.sample(random_state = 1, frac = 1).reset_index(drop=True)
poke_df.head()



,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary
0,Beedrill,Bug,Poison,395,65,90,40,45,80,75,3,False
1,Kingler,Water,NaN,475,55,130,115,50,50,75,2,False
2,Golem,Rock,Ground,495,80,120,130,55,65,45,3,False
3,Pidgeotto,Normal,Flying,349,63,60,55,50,50,71,2,False
4,Ditto,Normal,NaN,288,48,48,48,48,48,48,1,False


In [11]:
# muestra las columnas del DataFrame
poke_df.columns

Index(['Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Stage', 'Legendary'],
      dtype='object')

### Show the different type 1 present in the dataset

In general, there is no generic module or function to map and transform these features into numeric representations based on order automatically. Hence we can use a custom encoding\mapping scheme based on a dictionary.

In [12]:
# escribe un diccionario que mapee el Type 1 con un número asociado a cómo es de bueno el Type 1.
# Es decir, presupón que se pueden ordenar esas etiquetas.
# Usa DataFrame['Type 1'].unique() para seleccionar esos valores en ese orden y asignarles 1,2,3...
# Por ejemplo: 'Bug' se corresponde con 1, 'Water' se corresponde con 2...

# poke_df['Type 1'].unique()

type_1_map = {'Bug':1, 'Water':2, 'Rock':3, 'Normal':4, 'Fighting':5, 'Grass':6, 'Poison':7,
       'Fire':8, 'Ghost':9, 'Fairy':10, 'Electric':11, 'Dragon':12, 'Ground':13,
       'Psychic':14, 'Ice':15}


# mapea los valores en el DataFrame en una columna que se llame 'type_1_num'
# haz un head()

poke_df['type_1_num'] = poke_df['Type 1'].map(type_1_map)
poke_df.head()


,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary,type_1_num
0,Beedrill,Bug,Poison,395,65,90,40,45,80,75,3,False,1
1,Kingler,Water,NaN,475,55,130,115,50,50,75,2,False,2
2,Golem,Rock,Ground,495,80,120,130,55,65,45,3,False,3
3,Pidgeotto,Normal,Flying,349,63,60,55,50,50,71,2,False,4
4,Ditto,Normal,NaN,288,48,48,48,48,48,48,1,False,4


# Encoding Categorical Features

## One-hot Encoding Scheme

In [13]:
poke_df[['Name', 'Stage', 'Legendary']].iloc[4:10]

,Name,Stage,Legendary
4,Ditto,1,False
5,Primeape,2,False
6,Aerodactyl,1,False
7,Vileplume,3,False
8,Nidorina,2,False
9,Starmie,2,False


In [14]:
# usa LabelEncoder

from sklearn.preprocessing import LabelEncoder

# transform and map pokemon Type 1 with LabelEncoder
# el método zip te puede ayudar
le_pokemon = preprocessing.LabelEncoder()
le_pokemon.fit(poke_df['Type 1'])

le_pokemon.classes_
le_pokemon.transform(le_pokemon.classes_)

poke_df['Type 1 zip'] = poke_df['Type 1'].map(dict(zip(le_pokemon.classes_, le_pokemon.transform(le_pokemon.classes_))))


# transform and map pokemon legendary status with Label Encoder



In [15]:
# Otra forma más sencilla utilizando transform
# ¡Para esto vale fit y transform!
# Muchas transformaciones se dividen en fit (ajusta los parámetros de la transformación)
# y en transform (aplica los cambios)

le_pokemon.fit(poke_df['Type 1'])
poke_df['Type 1 transformed'] = le_pokemon.transform(poke_df['Type 1'])

In [16]:
#haz un head()
poke_df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary,type_1_num,Type 1 zip,Type 1 transformed
0,Beedrill,Bug,Poison,395,65,90,40,45,80,75,3,False,1,0,0
1,Kingler,Water,NaN,475,55,130,115,50,50,75,2,False,2,14,14
2,Golem,Rock,Ground,495,80,120,130,55,65,45,3,False,3,13,13
3,Pidgeotto,Normal,Flying,349,63,60,55,50,50,71,2,False,4,10,10
4,Ditto,Normal,NaN,288,48,48,48,48,48,48,1,False,4,10,10


In [17]:
# comprobamos que la codificación del método es alfabética
poke_df.sort_values('Type 1 zip')['Type 1'].unique()

array(['Bug', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Ghost',
       'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock',
       'Water'], dtype=object)

In [18]:
# haz un head()
poke_df.head()

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,Legendary,type_1_num,Type 1 zip,Type 1 transformed
0,Beedrill,Bug,Poison,395,65,90,40,45,80,75,3,False,1,0,0
1,Kingler,Water,NaN,475,55,130,115,50,50,75,2,False,2,14,14
2,Golem,Rock,Ground,495,80,120,130,55,65,45,3,False,3,13,13
3,Pidgeotto,Normal,Flying,349,63,60,55,50,50,71,2,False,4,10,10
4,Ditto,Normal,NaN,288,48,48,48,48,48,48,1,False,4,10,10


The features Type 1 zip and Legendary_zip now depict the numeric representations of our categorical features. Let’s now apply the one-hot encoding scheme on these features. Apply the get_dummies() method.

In [19]:
# encode Type 1 labels using one-hot encoding scheme
one_hot_df_type_1 = pd.get_dummies(poke_df['Type 1'], prefix = 'Type_1')

# encode legendary status labels using one-hot encoding scheme
one_hot_df_legendary = pd.get_dummies(poke_df['Legendary'], prefix = 'Legendary')


In [20]:

# compruebo que solo hay 4 pokemon legendarios
sum(one_hot_df_legendary['Legendary_True'])

4

In [21]:
# concatena el DataFrame original con la codificación de Type 1 y de Legendary 




Consider you built this encoding scheme on your training data and built some model and now you have some new data which has to be engineered for features before predictions as follows.

In [22]:
'''

new_poke_df = pd.DataFrame([['PikaZoom', 'Bug', True], 
                           ['CharMyToast', 'Water', False]],
                           columns=['Name', 'Type 1', 'Legendary'])
new_poke_df
'''

"\n\nnew_poke_df = pd.DataFrame([['PikaZoom', 'Bug', True], \n                           ['CharMyToast', 'Water', False]],\n                           columns=['Name', 'Type 1', 'Legendary'])\nnew_poke_df\n"

In [23]:
# usando fit() y transform(), añade Type1_Label y Lgnd_Label en el DataFrame



You can leverage scikit-learn’s excellent API here by calling the transform(…) function of the previously build LabeLEncoder objects on the new data.

## Dummy Coding Scheme

Let’s try applying dummy coding scheme on Pokémon Type 1 by dropping the first level binary encoded feature (Type 1 = Bug).


In [34]:
# haz un get_dummies para una codificación dummy
# muestra las filas desde la 4 hasta la 9 (incluida)

dummy_features = pd.get_dummies(poke_df, columns=['Type 1'], drop_first=True)
dummy_features.iloc[4:10]


,Name,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Stage,...,Type 1_Fire,Type 1_Ghost,Type 1_Grass,Type 1_Ground,Type 1_Ice,Type 1_Normal,Type 1_Poison,Type 1_Psychic,Type 1_Rock,Type 1_Water
4,Ditto,NaN,288,48,48,48,48,48,48,1,...,0,0,0,0,0,1,0,0,0,0
5,Primeape,NaN,455,65,105,60,60,70,95,2,...,0,0,0,0,0,0,0,0,0,0
6,Aerodactyl,Flying,515,80,105,65,60,75,130,1,...,0,0,0,0,0,0,0,0,1,0
7,Vileplume,Poison,490,75,80,85,110,90,50,3,...,0,0,1,0,0,0,0,0,0,0
8,Nidorina,NaN,365,70,62,67,55,55,56,2,...,0,0,0,0,0,0,1,0,0,0
9,Starmie,Psychic,520,60,75,85,100,85,115,2,...,0,0,0,0,0,0,0,0,0,1


If you want, you can also choose to drop the last level binary encoded feature

In [25]:
# haz un fit() de Type 1 y mira las clases que aparecen



In [26]:
# haz un dummies sin eliminar ninguna columna que se obtenga solo de Type 1
# haz un head()



In [27]:
# comprueba en la codificación con la columna eliminada (dummy)
# pista: isin te puede ayudar



In [28]:
# comprueba qué hace el signo ~



In [29]:
# haz una lectura en el DataFrame con la última sentencia que emplea ~



In [30]:
# asígnalo a una variable y muestra un head




## Feature Hashing scheme

Find the number of different 'Genre' in the dataset.

In [31]:
# Usa vgsales.csv, léelo y haz un head()




In [32]:
# print('Total game genres: ' + str(len(df_videojuegos.Genre.unique())))
# print(df_videojuegos.Genre.sort_values().unique())

### We can see that there are a total of 12 genres of video games. If we used a one-hot encoding scheme on the Genre feature, we would end up having 12 binary features. Instead, we will now use a feature hashing scheme by leveraging scikit-learn’s FeatureHasher class, which uses a signed 32-bit version of the Murmurhash3 hash function. We will pre-define the final feature vector size to be 6 in this case.